# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import yfinance as yf
import pandas as pd
import math
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~500 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [3]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [4]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        'Market Capitalization': [data['marketCap']],
        'Number of Shares to Buy': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price Market Capitalization Number of Shares to Buy
0        A       125.05           36936769536                     N/A
1      AAL        15.98           10440724480                     N/A
2      AAP        72.03            4281744128                     N/A
3     AAPL       191.17         3006855643136                     N/A
4     ABBV       149.05          262967443456                     N/A
..     ...          ...                   ...                     ...
498    YUM       135.62           37985398784                     N/A
499    ZBH       125.79           26263443456                     N/A
500   ZBRA       249.55           12834432000                     N/A
501   ZION        37.41            5542067200                     N/A
502    ZTS       180.88           83586818048                     N/A

[503 rows x 4 columns]


In [5]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,125.05,36936769536,N/A
1,AAL,15.98,10440724480,N/A
2,AAP,72.03,4281744128,N/A
3,AAPL,191.17,3006855643136,N/A
4,ABBV,149.05,262967443456,N/A
...,...,...,...,...
498,YUM,135.62,37985398784,N/A
499,ZBH,125.79,26263443456,N/A
500,ZBRA,249.55,12834432000,N/A
501,ZION,37.41,5542067200,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [6]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [7]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

19880.715705765408

In [8]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,125.05,36936769536,158
1,AAL,15.98,10440724480,1244
2,AAP,72.03,4281744128,276
3,AAPL,191.17,3006855643136,103
4,ABBV,149.05,262967443456,133
...,...,...,...,...
498,YUM,135.62,37985398784,146
499,ZBH,125.79,26263443456,158
500,ZBRA,249.55,12834432000,79
501,ZION,37.41,5542067200,531


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [9]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

## Formatting Our Excel Output

In [10]:
bg_color = '#0A0A23'
font_color = '#FFFFFF'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

In [11]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

In [12]:
writer.close()